<a id = main></a>
# Scraping website with multiple pages
This notebook is created to scrap 2020 FFIEC Census Demographic Reports from FFIEC website
- [**Import Packages**](#packages)
- [**Create Functions to Retrieve Census Demographic Reports by County**](#function)
- [**An Example of Application of Functions Crated**](#application)
    - [**Retrieve All FIPs From Wikipedia**](#fips)
    - [**Retrieve Census Demographic Reports of All Counties**](#censusreport)

<a id = packages></a>
## Import Packages
[**Go Back to Main Contents**](#main)

In [1]:
from lxml import html
import requests
import pandas as pd
from time import sleep
from random import randint
import dill
import re
# dill.load_session('ffiec.db')

In [2]:
fips = pd.read_pickle('fips.pkl')
all_states = pd.read_pickle('all_states.pkl')

<a id = function></a>
## Create Functions to Retrieve Census Demographic Reports by County
Create functions to find all census demographic reports for all states and counties
<br><br>
[**Go Back to Main Contents**](#main)

#### Function 1 
A function that allows us to pass different parameters to change the url
<br><br>
[**Go Back**](#function)

In [6]:
def create_url(county,state,page=1):
    url="https://www.ffiec.gov/census/report.aspx?year=2020&county={}&tract=ALL&state={}&report=demographic&page={}".format(county,state,page)
    return url

#### Function 2
A function that searches for the maximum number of pages of this report
<br><br>
[**Go Back**](#function)

In [7]:
def find_max_page(url):
    # Parse the main url and find the section says "Page"
    r = requests.get(url)
    parser = html.fromstring(r.content)
    
    find_page = parser.xpath("//*[contains(text(),'Page')]")

    # Set pages = 1 if there is only one page for the county, otherwise search for the maximum page number
    try:
        find_page[0].text_content()
        # Take the last string component from the string with "Page" since that is the maximum page number
        pages = int(find_page[0].text_content().split()[-1])
    except:
        pages = 1
    
    return pages

#### Function 3
A function that loops through all pages of the census demographic report and store data in one data frame
<br><br>
[**Go Back**](#function)

In [46]:
def parse_and_save(county,state):
    # Check if url is valid
    url_invalid = create_url(county,state)
    r_invalid = requests.get(url_invalid)
    parser_invalid = html.fromstring(r_invalid.content)
    invalid = parser_invalid.xpath("//*[@id='Report1_lblERR']")
    try:
        if 'Invalid' in str(invalid[0].text_content()): # If url is invalid, then return 'Invalid'
            Invalid = 'Invalid'
            return Invalid
    except:
        pages = find_max_page(create_url(county,state)) # Find the maximum number of pages

        for page in range(1,int(pages)+1):
            url = create_url(county,state,page) # Change url as we move from page to page
            r = requests.get(url)
            parser = html.fromstring(r.content)
            tb = parser.xpath("//table[@id='Report1_dgReportDemographic']//tr") # Find the section where the demographic table locates
            # Paser table header if we are on the first page and initial the output data frame
            if (page == 1):
                cols = ['State','County','Page']
                for col in tb[0]:
                    cols.append(col.text_content())
                df = pd.DataFrame([cols])

            for i in range(1,len(tb)): # Loop through each row since the second row to exclude the header
                if len(tb[i]) != 12:
                    break
                row = [state,county,page] # Initialize a list to store elements in a row
                for j in range(0,len(tb[i])): # Loop through each element in a row
                    element = str(tb[i][j].text_content())
                    try:
                        element = float(element.replace(',','').replace('$',''))
                    except:
                        pass
                    row.append(element)
                df.loc[len(df)] = row

            sleep(randint(5,10)) # Control the scrapping rate - avoid stressing out the server and being banned

        df.columns = df.loc[0] # Set the first row as table header
        df = df[1:] # Remove the first row
        df.reset_index(drop = True,inplace = True)
        return df

<a id = application></a>
## An Example of Application of Functions Created
In this example, I am going to loop through all states and counties in the United States and retrieve their census demographic reports, and eventually store them in a data frame called 'all_states'
<br><br>
[**Go Back to Main Contents**](#main)

<a id = fips></a>
#### Step 1: Retrieve All FIPs from Wikipedia
Retrieve FIPs for states and counties in the U.S. from Wikipedia. Data is stored in a data frame called 'fips'
<br><br>
[**Go Back**](#application)

In [ ]:
# Retrieve and parse Wikipedia page, and find the table to be stored
fips_url = "https://en.wikipedia.org/wiki/List_of_United_States_FIPS_codes_by_county"
fips_r = requests.get(fips_url)
fips_parser = html.fromstring(fips_r.content)

fips_tb = fips_parser.xpath("//*[@id='mw-content-text']/div[1]/table[2]//tr")

In [ ]:
# Loop through all rows in the table and store them in a data frame

fips = pd.DataFrame([['FIPS','County','State']]) # Create an empty data frame with headers
state = "Alabama" # Initial state is Alabama

for i in range(1,len(fips_tb)):
    row = fips_tb[i].text_content().replace('\xa0','').split('\n') 
    row = [re.sub("[\(\[].*?[\)\]]","",str(i)) for i in row if i] # Remove empty strings in the list
    
    # If there are 3 or more elements in a row, then state is updated
    if len(row) >= 3:
        state = row[2]
    else:
        state = str(state)
        row.append(state)
    
    fips.loc[len(fips)] = row
    
fips.columns = fips.loc[0] # Set the first row as table header
fips = fips[1:] # Remove the first row
fips.reset_index(drop = True,inplace = True)

In [49]:
pd.set_option('display.max_rows', 3242)
fips

,FIPS,County,State
0,01001,Autauga County,Alabama
1,01003,Baldwin County,Alabama
2,01005,Barbour County,Alabama
3,01007,Bibb County,Alabama
4,01009,Blount County,Alabama
5,01011,Bullock County,Alabama
6,01013,Butler County,Alabama
7,01015,Calhoun County,Alabama
8,01017,Chambers County,Alabama
9,01019,Cherokee County,Alabama


<a id = censusreport></a>
#### Step 2: Retrieve Census Demographic Reports of All Counties
Loop through all states and counties and retrieve the corresponding census demographic reports, and store them in 'all_states' data frame
<br><br>
[**Go Back**](#application)

In [ ]:
# Initial the data frame to store all census reports
all_states=pd.DataFrame()

In [48]:
# Loop through all states and counties in fips data frame and retrieve the corresponding reports
for i in range(0,len(fips)):
    state = fips.loc[i][0][0:2]
    county = fips.loc[i][0][2:5]
    state_name = fips.loc[i][2]
    county_name = fips.loc[i][1]
    
    print(str(i) + ', ' + state + ' ' + state_name + ', ' + county + ' ' + county_name)
    try:
        if parse_and_save(county = county,state = state) == 'Invalid':
            print(parse_and_save(county = county,state = state))
            continue
    except:
        pass
    df = parse_and_save(county = county,state = state)
    df['State_Name'] = state_name
    df['County_Name'] = county_name
    all_states = pd.concat([all_states,df],ignore_index = True)
    
    sleep(randint(5,10)) # Control the scrapping rate - avoid stressing out the server and being banned

286574  U.S. Minor Outlying Islands, 050 Baker Island
Invalid
286674  U.S. Minor Outlying Islands, 100 Howland Island
Invalid
286774  U.S. Minor Outlying Islands, 150 Jarvis Island
Invalid
286874  U.S. Minor Outlying Islands, 200 Johnston Atoll
Invalid
286974  U.S. Minor Outlying Islands, 250 Kingman Reef
Invalid
287074  U.S. Minor Outlying Islands, 300 Midway Islands
Invalid
287174  U.S. Minor Outlying Islands, 350 Navassa Island
Invalid
287274  U.S. Minor Outlying Islands, 400 Palmyra Atoll
Invalid
287374  U.S. Minor Outlying Islands, 450 Wake Island
Invalid
287449 Utah, 001 Beaver County
287549 Utah, 003 Box Elder County
287649 Utah, 005 Cache County
287749 Utah, 007 Carbon County
287849 Utah, 009 Daggett County
287949 Utah, 011 Davis County
288049 Utah, 013 Duchesne County
288149 Utah, 015 Emery County
288249 Utah, 017 Garfield County
288349 Utah, 019 Grand County
288449 Utah, 021 Iron County
288549 Utah, 023 Juab County
288649 Utah, 025 Kane County
288749 Utah, 027 Millard County


307953  Washington, 053 Pierce County
308053  Washington, 055 San Juan County
308153  Washington, 057 Skagit County
308253  Washington, 059 Skamania County
308353  Washington, 061 Snohomish County
308453  Washington, 063 Spokane County
308553  Washington, 065 Stevens County
308653  Washington, 067 Thurston County
308753  Washington, 069 Wahkiakum County
308853  Washington, 071 Walla Walla County
308953  Washington, 073 Whatcom County
309053  Washington, 075 Whitman County
309153  Washington, 077 Yakima County
309254 West Virginia, 001 Barbour County
309354 West Virginia, 003 Berkeley County
309454 West Virginia, 005 Boone County
309554 West Virginia, 007 Braxton County
309654 West Virginia, 009 Brooke County
309754 West Virginia, 011 Cabell County
309854 West Virginia, 013 Calhoun County
309954 West Virginia, 015 Clay County
310054 West Virginia, 017 Doddridge County
310154 West Virginia, 019 Fayette County
310254 West Virginia, 021 Gilmer County
310354 West Virginia, 023 Grant County


In [50]:
all_states

,State,County,Page,Tract Code,Tract Income Level,Distressed or Under -served Tract,Tract Median Family Income %,2020 FFIEC Est. MSA/MD non-MSA/MD Median Family Income,2020 Est. Tract Median Family Income,2015 Tract Median Family Income,Tract Population,Tract Minority %,Minority Population,Owner Occupied Units,1- to 4- Family Units,State_Name,County_Name
0,01,001,1,201,Upper,No,122.93,65700,80765,72727,1948,12.58,245,507,724,Alabama,Autauga County
1,01,001,1,202,Middle,No,82.4,65700,54137,48750,2156,59.55,1284,433,785,Alabama,Autauga County
2,01,001,1,203,Middle,No,94.26,65700,61929,55766,2968,25.47,756,828,1327,Alabama,Autauga County
3,01,001,1,204,Middle,No,116.82,65700,76751,69114,4423,17.21,761,1345,1806,Alabama,Autauga County
4,01,001,1,205,Upper,No,127.74,65700,83925,75574,10763,31.54,3395,2255,3237,Alabama,Autauga County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75878,56,043,1,3.02,Middle,Yes*,91.93,79700,73268,66958,2566,23.81,611,799,1077,Wyoming,Washakie County
75879,56,043,1,9999.99,Middle,No,90.77,79700,72344,66113,8400,17.61,1479,2590,3743,Wyoming,Washakie County
75880,56,045,1,9511,Upper,No,120.81,79700,96286,87994,3442,6.36,219,1103,1724,Wyoming,Weston County
75881,56,045,1,9513,Middle,No,106.78,79700,85104,77775,3710,9.14,339,1227,1621,Wyoming,Weston County


In [51]:
# Save important data frames in this workspace
fips.to_pickle("./fips.pkl")
all_states.to_pickle("./all_states.pkl")